In [1]:

print("app file created")


app file created


In [2]:

import streamlit as st
import sqlite3
import jwt
import datetime

SECRET_KEY = "policynav_secret"

conn = sqlite3.connect("users.db", check_same_thread=False)
c = conn.cursor()
c.execute("""
CREATE TABLE IF NOT EXISTS users(
    username TEXT,
    email TEXT PRIMARY KEY,
    password TEXT,
    question TEXT,
    answer TEXT
)
""")
conn.commit()

st.set_page_config(page_title="PolicyNav Auth")

menu = ["Login", "Signup", "Forgot Password"]
choice = st.sidebar.selectbox("Menu", menu)

if choice == "Signup":
    st.title("Signup")

    username = st.text_input("Username")
    email = st.text_input("Email")
    password = st.text_input("Password", type="password")
    confirm = st.text_input("Confirm Password", type="password")
    question = st.selectbox(
        "Security Question",
        ["Pet name?", "Mother's maiden name?", "Favourite teacher?"]
    )
    answer = st.text_input("Security Answer")

    if st.button("Signup"):
        if not (username and email and password and confirm and answer):
            st.error("All fields required")
        elif password != confirm:
            st.error("Passwords do not match")
        else:
            c.execute(
                "INSERT OR REPLACE INTO users VALUES (?,?,?,?,?)",
                (username, email, password, question, answer)
            )
            conn.commit()
            st.success("Signup successful!")

elif choice == "Login":
    st.title("Login")

    email = st.text_input("Email")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        c.execute("SELECT username FROM users WHERE email=? AND password=?",
                  (email, password))
        user = c.fetchone()

        if user:
            token = jwt.encode({
                "user": email,
                "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=1)
            }, SECRET_KEY, algorithm="HS256")

            st.session_state["logged_in"] = True
            st.session_state["user"] = user[0]
            st.success("Login successful")
        else:
            st.error("Invalid credentials")

if st.session_state.get("logged_in"):
    st.title("Dashboard")
    st.write(f"Welcome {st.session_state['user']} 👋")

    if st.button("Logout"):
        st.session_state.clear()
        st.success("Logged out")

if choice == "Forgot Password":
    st.title("Forgot Password")

    email = st.text_input("Email")
    if st.button("Get Question"):
        c.execute("SELECT question, answer FROM users WHERE email=?", (email,))
        data = c.fetchone()

        if data:
            st.session_state["q"] = data[0]
            st.session_state["a"] = data[1]
            st.session_state["email"] = email
        else:
            st.error("Email not found")

    if "q" in st.session_state:
        st.write("Security Question:", st.session_state["q"])
        ans = st.text_input("Your Answer")
        new_pass = st.text_input("New Password", type="password")

        if st.button("Reset Password"):
            if ans == st.session_state["a"]:
                c.execute("UPDATE users SET password=? WHERE email=?",
                          (new_pass, st.session_state["email"]))
                conn.commit()
                st.success("Password updated")
            else:
                st.error("Wrong answer")


2026-02-15 15:26:20.014 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-15 15:26:20.016 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-15 15:26:20.020 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-15 15:26:20.021 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-15 15:26:20.024 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-15 15:26:20.025 WARNING streamlit.run